In [1]:
# TODO: Make all necessary imports.
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


Using:
	• TensorFlow version: 2.18.0
	• GPU device not found. Running on CPU


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

class Network(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        # URLs for the feature extractors
        URL1 = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5"
        URL2 = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

        # Define the feature extractors
        self.feature_extractor1 = hub.KerasLayer(URL1, trainable=False, input_shape=(224, 224, 3))
        self.feature_extractor2 = hub.KerasLayer(URL2, trainable=False, input_shape=(224, 224, 3))

        # Define fully connected layers
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))
        self.dropout1 = tf.keras.layers.Dropout(0.2)
        self.dense2 = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))
        self.dropout2 = tf.keras.layers.Dropout(0.2)
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001))

    def call(self, x):
        # Pass the input image through both feature extractors
        x1 = self.feature_extractor1(x)  # Output shape: (batch_size, features1)
        x2 = self.feature_extractor2(x)  # Output shape: (batch_size, features2)

        # Concatenate the outputs from both extractors
        x = tf.concat([x1, x2], axis=-1)

        # Pass the concatenated features through the dense layers
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)
        x = self.output_layer(x)
        return x
    
    def get_config(self):
        return {"num_classes": self.num_classes}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [9]:
# check this later
import tensorflow as tf
import tensorflow_hub as hub

class Network(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        # URLs for the feature extractors
        URL1 = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5"
        URL2 = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

        # Define the feature extractors
        self.feature_extractor1 = hub.KerasLayer(URL1, trainable=False, input_shape=(224, 224, 3))
        self.feature_extractor2 = hub.KerasLayer(URL2, trainable=False, input_shape=(224, 224, 3))

        # Define fully connected layers
        self.dense1 = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))
        self.dropout1 = tf.keras.layers.Dropout(0.2)
        self.dense2 = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))
        self.dropout2 = tf.keras.layers.Dropout(0.2)
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001))

    def call(self, x):
        # Pass the input image through both feature extractors
        x1 = self.feature_extractor1(x)  # Output shape: (batch_size, features1)
        x2 = self.feature_extractor2(x)  # Output shape: (batch_size, features2)

        # Concatenate the outputs from both extractors
        x = tf.concat([x1, x2], axis=-1)

        # Pass the concatenated features through the dense layers
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)
        x = self.output_layer(x)
        return x


In [7]:
import tensorflow as tf
import tensorflow_hub as hub

print("TensorFlow version:", tf.__version__)
print("TensorFlow Hub version:", hub.__version__)


TensorFlow version: 2.18.0
TensorFlow Hub version: 0.16.1


In [5]:
savedModel_directory = f'./multi-tensorflowhub-layers-model{int(t)}.keras'

loaded_model = tf.keras.models.load_model(
    './multi-tensorflowhub-layers-model1734443097.keras', 
    custom_objects={'Network': Network}
)
# reloaded_model = tf.keras.models.load_model('./1734433943.keras')


NameError: name 't' is not defined